In [1]:
import ee
import geemap
import ipywidgets as widgets
from datetime import datetime, timedelta

In [4]:
ee.Initialize()

Map = geemap.Map(center=[48.6, 37.46], zoom=8)
Map.add_basemap('OpenStreetMap')

# Styl KML
kml_file = r"C:\Users\slesi\Desktop\Russian_fortifications.kml"
style = {
    'color': 'red',
    'fillColor': 'red',
    'opacity': 0.8,
    'fillOpacity': 0.5
}
Map.add_kml(kml_file, layer_name='Fortyfikacje KML', style=style)

# Parametry bufora i wizualizacji
buffer_m = 5000
true_color_vis = {"min": 0, "max": 3000, "bands": ["B4", "B3", "B2"]}
years = range(2020, 2026)
months = range(1, 13)

# Maska chmur (SCL)
def mask_cloud(image):
    scl = image.select('SCL')
    cloud_mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10))
    return image.updateMask(cloud_mask)

# Funkcja generująca kolekcję czasową
def create_time_series(point):
    roi = point.buffer(buffer_m).bounds()
    images = []
    labels = []

    for year in years:
        for month in months:
            start = ee.Date.fromYMD(year, month, 1)
            end = start.advance(1, 'month')
            img = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterBounds(roi) \
                .filterDate(start, end) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                .map(mask_cloud) \
                .mean() \
                .clip(roi) \
                .set('label', f"{year}-{month:02}")
            images.append(img)
            labels.append(f"{year}-{month:02}")
    
    return ee.ImageCollection.fromImages(images), labels

# Przycisk i callback
load_button = widgets.Button(description="Załaduj obrazy Sentinel-2")

def on_button_click(b):
    if Map.user_roi is None:
        print("⚠️ Najpierw narysuj punkt (Toolbox → Draw → Point)")
        return
    
    geom = Map.user_roi
    point = geom.centroid()

    # Wyczyść wszystkie warstwy oprócz KML + basemap
    Map.layers = Map.layers[:2]

    ic, names = create_time_series(point)
    Map.ts_inspector(
        left_ts=ic,
        right_ts=ic,
        left_names=names,
        right_names=names,
        left_vis=true_color_vis,
        right_vis=true_color_vis,
    )
    coords = point.coordinates().getInfo()
    Map.setCenter(coords[0], coords[1], 10)

load_button.on_click(on_button_click)

# Wyświetl mapę i przycisk
display(load_button)
Map


Button(description='Załaduj obrazy Sentinel-2', style=ButtonStyle())

Map(center=[48.6, 37.46], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…